In [1]:
# Install dependencies
!pip install -q amplpy ampltools

# Google Colab & AMPL integration
MODULES, LICENSE_UUID = ["coin", 'gurobi', "cplex", "highs", "gokestrel"], "42fc7eb6-69aa-445d-b655-3ad24d836541"
from amplpy import tools
from ampltools import cloud_platform_name, ampl_notebook, register_magics

# instantiate AMPL object and register magics
if cloud_platform_name() is None:
    ampl = AMPL() # Use local installation of AMPL
else:
    ampl = tools.ampl_notebook(modules=MODULES, license_uuid=LICENSE_UUID, g=globals())

register_magics(ampl_object=ampl)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 24.7 MB/s eta 0:00:00
Licensed to Bundle #6741.7193 expiring 20241231: INFO 645 Prescriptive Analytics, Prof. Paul Brooks, Virginia Commonwealth University.


In [6]:
from scipy.optimize import linprog

# Coefficients of the objective function (negative because linprog minimizes)
c = [-0.12, -0.09]

# Coefficients of the inequality constraints (Ax <= b)
A = [
    [1, 1],       # Total investment constraint: x1 + x2 <= 50000
    [1, 0],       # Investment limit for Internet fund: x1 <= 35000
    [6, 4]        # Risk constraint: 6x1 + 4x2 <= 240000
]

# Bounds for the right-hand side of the inequality constraints
b = [50000, 35000, 240000]

# Bounds for the decision variables (x1 and x2 must be non-negative)
x_bounds = [(0, None), (0, None)]

# Solve the linear programming problem
result = linprog(c, A_ub=A, b_ub=b, bounds=x_bounds, method='highs')

# Output the results
if result.success:
    x1, x2 = result.x
    max_return = -result.fun
    print(f"Optimal investment in the Internet fund: ${x1:.2f}")
    print(f"Optimal investment in the Blue Chip fund: ${x2:.2f}")
    print(f"Maximum annual return: ${max_return:.2f}")
else:
    print("The optimization problem did not converge to a solution.")



Optimal investment in the Internet fund: $20000.00
Optimal investment in the Blue Chip fund: $30000.00
Maximum annual return: $5100.00


In [7]:
# Install dependencies
!pip install -q amplpy ampltools

# Google Colab & AMPL integration
MODULES, LICENSE_UUID = ["coin", 'gurobi', "cplex", "highs", "gokestrel"], "42fc7eb6-69aa-445d-b655-3ad24d836541"
from amplpy import tools
from ampltools import cloud_platform_name, ampl_notebook, register_magics

# instantiate AMPL object and register magics
if cloud_platform_name() is None:
    ampl = AMPL() # Use local installation of AMPL
else:
    ampl = tools.ampl_notebook(modules=MODULES, license_uuid=LICENSE_UUID, g=globals())

register_magics(ampl_object=ampl)

# Define model
ampl.eval ('''
reset;

# Parameters
param max_investment := 50000; # Maximum total investment
param max_internet_investment := 35000; # Maximum investment in the Internet fund
param max_risk := 240000; # Maximum risk tolerance
param return_internet := 0.12; # Annual return of the Internet fund
param return_bluechip := 0.09; # Annual return of the Blue Chip fund
param risk_internet := 6; # Risk per $1,000 for the Internet fund
param risk_bluechip := 4; # Risk per $1,000 for the Blue Chip fund

# Decision Variables
var x1 >= 0; # Amount invested in the Internet fund
var x2 >= 0; # Amount invested in the Blue Chip fund

# Objective Function: Maximize total annual return
maximize TotalReturn: return_internet * x1 + return_bluechip * x2;

# Constraints
subject to
  InvestmentLimit: x1 + x2 <= max_investment; # Total investment constraint
  InternetFundLimit: x1 <= max_internet_investment; # Limit for the Internet fund
  RiskLimit: risk_internet * x1 + risk_bluechip * x2 <= max_risk; # Risk tolerance constraint

''')


Licensed to Bundle #6741.7193 expiring 20241231: INFO 645 Prescriptive Analytics, Prof. Paul Brooks, Virginia Commonwealth University.


In [8]:
# Expand to confirm AMPL model syntax is correct
ampl.eval('expand;')

# Set solver option and solve
ampl.setOption('solver', 'cbc')
ampl.solve()

maximize TotalReturn:
	0.12*x1 + 0.09*x2;

subject to InvestmentLimit:
	x1 + x2 <= 50000;

subject to InternetFundLimit:
	x1 <= 35000;

subject to RiskLimit:
	6*x1 + 4*x2 <= 240000;

cbc 2.10.10: cbc 2.10.10: optimal solution; objective 5100
0 simplex iterations


In [11]:

# Get and print results
obj = ampl.getObjective('TotalReturn')
print("\nTotal maximum return is: ", obj.value(), "\n")
print("Optimal investments:")
ampl.display('x1', 'x2')


Total maximum return is:  5100.0 

Optimal investments:
x1 = 20000
x2 = 30000

